## Python imports

In [ ]:
import requests
import json
import os
import sys
from random import randint
import urllib3
urllib3.disable_warnings()

### _Load Django into Jupyter_

In [ ]:
sys.path.append('/bookstore')
sys.path.append('/bookstore/project')
os.environ['DJANGO_SETTINGS_MODULE'] = 'project.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'
import django
django.setup()

In [ ]:
def get_token(url, username, password):
    payload = {'username': username, 'password': password}
    headers = {'Content-type': 'application/json'}
    return requests.post(f'{url}/token/', json=payload, headers=headers).json()['access']

In [ ]:
url = 'http://bookstore:8000/api'
token = get_token(url, 'carlos.carrot', 'secret123')
headers = {'Content-type': 'application/json', 'Authorization': f'Bearer {token}'}
books = requests.get(f'{url}/book', headers=headers).json()
rand_book = books[randint(0, len(books)-1)]
rand_book['title'] += " Appeltaart"
book = requests.put(rand_book['url'], json=rand_book, headers=headers).json()
book

In [ ]:
from bookstore.models import *

my_book = Book.objects.get(id=book['id'])

for h in my_book.history.all():
    print(h)

my_book.history.all()

In [ ]:
# requests.get(f'{url}/book?author=Ray%20Bradbury', headers=headers).json()

## Challenge #8
### Logging to Splunk
Create a token and post a log message

In [ ]:
with open("/templates/splunk.py", "r+") as file:
    data = file.read()
    if not os.path.exists("/bookstore/bookstore"):
        os.makedirs("/bookstore/bookstore")
    with open("/bookstore/bookstore/splunk.py", "w") as file:
        write_file = file.write(data)

# print(open("/bookstore/bookstore/splunk.py").read())

In [ ]:
# import os

# os.listdir('/bookstore/bookstore')

In [ ]:
from bookstore.splunk import *

app = 'bookstore'

index = get_splunk_index(app)
print(json.dumps(index, indent=4, sort_keys=True))

input_http, splunk_token = get_splunk_input_http(app)
print(json.dumps(input_http, indent=4, sort_keys=True))

## Check Collector

In [ ]:
url = "https://splunk:8088/services/collector"
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Splunk {splunk_token}'
}

response = requests.get(f"{url}/health", headers=headers, verify=False).json()
print(json.dumps(response, indent=4, sort_keys=True))

## Upload log entries

In [ ]:
import random

for i in range(0,4):
    users = ["Anna", "Bob", "Carlos"]
    dict = { 'message' : "Dummy test python client log", 'user': random.choice(users)}
    log = splunklog('bookstore', dict)
    print(log)

In [ ]:
from time import sleep

endpoint = "https://splunk:8089"
auth = ('admin', 'secret123')
data = {
    'search': f'search index={app}_index',
    'output_mode':'json'
}
response = requests.post(f"{endpoint}/services/search/jobs", data=data, auth=auth, verify=False).json()
sid=response['sid']

while True:
    response = requests.get(f"{endpoint}/services/search/jobs/{sid}", data=data, auth=auth, verify=False).json()
    if response['entry'][0]['content']['isDone']:
        break
    else:
        sleep(1)

url = f"{endpoint}{response['entry'][0]['links']['events']}"
response = requests.get(url, data=data, auth=auth, verify=False).json()
for log in response['results']:
    print(json.dumps(json.loads(log['_raw']), indent=4, sort_keys=True))